In [ ]:
import sys
import csv
import pandas as pd
import numpy as np

import cartopy
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
from cartopy.feature import ShapelyFeature
import cartopy.io.shapereader as shpreader
import matplotlib.colors as mcolors

import holoviews as hv
hv.extension('bokeh')

import bokeh.io
bokeh.io.output_notebook()



## Plot oil and gas wells using cartopy

Import the oil and gas well data 

In [ ]:
arkog = shpreader.Reader("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/AR_OG_AllWells_05032021/ArkansasOilandGasWells_05032021.shp")

In [ ]:
colog = shpreader.Reader("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/CO_OilandGas_WELLS_SHP_110321/Wells.shp")

In [ ]:
kansog = shpreader.Reader("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/KS_SWD_WELLS_Volumes_End_2018_With_Excel_KML/KS_SWD_WELLS_Volumes_End_2018.shp")

In [ ]:
missog = shpreader.Reader("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/MO_OG_AllWells_04142021/MissouriWells.shp")

In [ ]:
newmexlog = shpreader.Reader("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/NM_03152022_OGWells/WellsGIS/WellsGIS.shp")


In [ ]:
oklaog = shpreader.Reader("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/OK_ClassII_2011_2018_Volumes_WithExcel/OK_ClassII_2011_2018_Volumes.shp")


In [ ]:
texasog = pd.read_csv("/Users/arianatribby/Documents/oklahoma_c3_c2/OGWells/TX_Prod5-8-2021/TX_OGwells_05082021.csv")

In [ ]:
texasog.head()

In [ ]:
oklaog.records()

In [ ]:
next(oklaog.records()).geometry

In [ ]:
next(oklaog.records()).attributes

Stack overflow is saying to take a look at the .prj file in the folder, which contains info about units for the geometries. Here is all the file contained: 


GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]




In [ ]:
next(oklaog.records()).geometry.centroid.xy 

so those units look like lat/lon

In [ ]:
fig = plt.figure()
# ax = fig.add_axes((0, 0, 1, 1), projection=ccrs.PlateCarree())
ax = plt.axes(projection=ccrs.PlateCarree())

ax.stock_img()


ax.set_extent([-110,-85,25,45])

# # Create a feature for States/Admin 1 regions at 1:50m from Natural Earth
# states_provinces = cfeature.NaturalEarthFeature(
#     category='cultural',
#     name='admin_1_states_provinces_lines',
#     scale='50m',
#     facecolor='none')

SOURCE = 'Natural Earth'
LICENSE = 'public domain'

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)
# ax.add_feature(states_provinces, edgecolor='gray')
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')

# plt.text(-97.56, 36.69, 'Lamont',
#          horizontalalignment='right',
#          transform=ccrs.Geodetic())

# not sure why the following didn't work: 
# shape_feature = ShapelyFeature(oklaog.geometries(),ccrs.PlateCarree(), 
#                                edgecolor='black')
# ax.add_feature(shape_feature, facecolor='blue')



# the add geometries method below doesn't read from disk so very slow .................
# ax.add_geometries([next(oklaog.records()).geometry], ccrs.PlateCarree())
# ax.add_geometries([next(oklaog.records()).geometry], ccrs.PlateCarree(),
#                   facecolor='blue', alpha=0.4)
# ax.add_geometries([next(oklaog.records()).geometry], ccrs.epsg(26910))
# ax.add_feature(next(oklaog.records()).geometry, ccrs.PlateCarree(),
#                   facecolor='blue', alpha=0.4)

for pp in oklaog.records():
#     col=colour[1]
#     if pp.attributes['ForestType'] == 'OG':
#         col = colour[0]
#     elif pp.attributes['ForestType'] == 'ML':
#         col=colour[2]
    ax.plot(pp.geometry.coords.xy[0],pp.geometry.coords.xy[1], marker='o', 
            mfc='#8B0000',
            mec='#8B0000', mew=0.2, markersize=1)

# use mcolors.CSS4_COLORS command to get the number id for colors



plt.show()

Plot all Wells at once for the paper 

But some states have their geometries in units of meters. I know you can approximate lat/lon as meter but was not wanting to do that here for fear of introducing additonal uncertainty. Luckily the lat/lon was part of the attributes section. For future if need to convert, here are some (cumbersome) ways of doing it: 

BEST SOLUTIPN is using pyproj, simple enough but not sure if you can do this all at once on arrays of lat/lon https://gis.stackexchange.com/questions/78838/converting-projected-coordinates-to-lat-lon-using-python but you need to double check what the coordinate is, the 3857 may be mercator projection.  
```
from pyproj import Proj, transform

inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')
x1,y1 = -11705274.6374,4826473.6922
x2,y2 = transform(inProj,outProj,x1,y1)
print x2,y2
```
https://stackoverflow.com/questions/56662941/cartopy-convert-point-from-axes-coordinates-to-lat-lon-coordinates 
https://climate-cms.org/posts/2021-10-01-pyproj.html
not changing units, but using different projection (but I tried using different projection while units were in meters and that didn't plot) https://uoftcoders.github.io/studyGroup/lessons/python/cartography/lesson/ 


In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

ax.stock_img()
ax.set_extent([-110,-85,25,45])

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')

ax.scatter(texasog.LONG83.values, texasog.LAT83.values, marker='o', color='#8B0000', s=1, alpha=.5)   


for pp in oklaog.records():
    ax.plot(pp.geometry.coords.xy[0],pp.geometry.coords.xy[1], marker='o', 
            mfc='#8B0000',mec='#8B0000', mew=0.2, markersize=1, alpha=.5)

for pp in arkog.records():
    # the geometry is in units of meters, but the attributes have the lat/lon location
    ax.plot(pp.attributes['longitude'],pp.attributes['latitude'], marker='o', 
            mfc='#8B0000',mec='#8B0000', mew=0.2, markersize=1, alpha=.5)
    
for pp in colog.records():   
    # the geometry is in units of meters, but the attributes have the lat/lon location
    ax.plot(pp.attributes['Longitude'],pp.attributes['Latitude'], marker='o', 
            mfc='#8B0000',mec='#8B0000', mew=0.2, markersize=1, alpha=.5)

for pp in kansog.records():
     ax.plot(pp.geometry.coords.xy[0],pp.geometry.coords.xy[1], marker='o', 
            mfc='#8B0000',mec='#8B0000', mew=0.2, markersize=1, alpha=.5)

for pp in missog.records():
     ax.plot(pp.geometry.coords.xy[0],pp.geometry.coords.xy[1], marker='o', 
            mfc='#8B0000',mec='#8B0000', mew=0.2, markersize=1, alpha=.5)

for pp in newmexlog.records():
     ax.plot(pp.geometry.coords.xy[0],pp.geometry.coords.xy[1], marker='o', 
            mfc='#8B0000',mec='#8B0000', mew=0.2, markersize=1, alpha=.5)

plt.savefig("allwells_for_emissionsforstilt.png")
plt.show()

Plot the emissions from emissions_forstilt.ipynb

In [ ]:
emiss = shpreader.Reader('/Users/arianatribby/git/oklahoma_propane/data/processing/wellhead_emiss_for_footprint.shp')

In [ ]:
next(emiss.records()).attributes

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

ax.stock_img()
ax.set_extent([-115,-80,25,50])

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=2, color='gray', alpha=0.5, linestyle='--')

emissgrid = cartopy.feature.ShapelyFeature(emiss.geometries(),ccrs.PlateCarree(), facecolor='none')
ax.add_feature(emissgrid, edgecolor='r', linewidth=0.5)

# neither of these worked 
# ax.add_geometries(emiss.geometries(), crs=ccrs.PlateCarree(), facecolor='blue', edgecolor='red', alpha=0.8)
# for em in emiss.records():
# #     name = em.attributes['n_wells']
#     ax.add_geometries([em.geometry], ccrs.PlateCarree(), facecolor='k', alpha=0.4)


# for pp in oklaog.records():
#     ax.plot(pp.geometry.coords.xy[0],pp.geometry.coords.xy[1], marker='o', 
#             mfc='#8B0000',mec='#8B0000', mew=0.2, markersize=1, alpha=.5)


# plt.savefig("allwells_for_emissionsforstilt.png")
plt.show()

why do the geometries extend across the entire grid? 
these instructions were helping you https://notebook.community/hetland/python4geosciences/materials/ST_shapefiles 

I double checked that emissions were good in the python file where it came from 